In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly
chart_studio.tools.set_credentials_file(username='tom.hd', api_key='ofaW3SZfg0TMqQ6oiVVN')
chart_studio.tools.set_config_file(world_readable=True, sharing='public')
import chart_studio.plotly as py

In [2]:
df_energy = pd.read_csv('data/Energy_Transition.csv')
df_energy.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Technology,Energy_Type,Unit,Source,CTS_Name,...,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020
0,1,Advanced Economies,NaN,AETMP,Electricity Generation,Bioenergy,Total Renewable,Gigawatt-hours (GWh),International Renewable Energy Agency (IRENA) ...,NaN,...,208500.849,224904.936,239773.166,252341.180,264181.433,274068.035,283583.581,287858.956,293917.663,296348.479
1,2,Advanced Economies,NaN,AETMP,Electricity Installed Capacity,Bioenergy,Total Renewable,Megawatt (MW),International Renewable Energy Agency (IRENA) ...,NaN,...,41776.427,43690.635,46115.653,48575.397,50782.092,53205.387,55580.629,58470.495,59285.425,59698.238
2,3,Advanced Economies,NaN,AETMP,Electricity Generation,Fossil fuels,Total Non-Renewable,Gigawatt-hours (GWh),International Renewable Energy Agency (IRENA) ...,NaN,...,6184941.257,6192108.018,6103628.918,5966341.171,5895296.029,5862708.121,5724991.355,5780558.599,5552191.520,5239862.113
3,4,Advanced Economies,NaN,AETMP,Electricity Installed Capacity,Fossil fuels,Total Non-Renewable,Megawatt (MW),International Renewable Energy Agency (IRENA) ...,NaN,...,1548870.635,1549434.445,1536047.763,1543835.825,1520799.171,1507492.990,1504881.823,1519986.073,1496654.330,1487980.682
4,5,Advanced Economies,NaN,AETMP,Electricity Generation,Geothermal energy,Total Renewable,Gigawatt-hours (GWh),International Renewable Energy Agency (IRENA) ...,NaN,...,37192.016,38076.418,38936.706,40294.245,40949.973,40915.479,41096.979,41910.671,41891.049,42477.806


In [3]:
df_energy = df_energy.drop(['ObjectId', 'ISO3', 'Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor'], axis=1)
df_energy.rename({'Energy_Type': 'Energy Type'}, axis=1, inplace=True)
df_energy['Technology'].replace({'Hydropower (excl. Pumped Storage)': 'Hydropower', "Other non-renewable energy": "Other non-renewable", "Pumped storage": "Other non-renewable", "Marine energy": "Other renewable", "Geothermal energy": "Other renewable"}, inplace=True)
df_energy['Country'].replace({'China, P.R.: Mainland': 'China', 'Korea, Rep. of': 'Korea', 'Russian Federation': 'Russia' }, inplace=True)

# regular expressions to format the years column names and units
for col in df_energy.columns:
    if re.match(r'F\d{4}', col):
        new_col = re.sub(r'F(\d{4})', r'\1', col)
        df_energy.rename(columns={col: new_col}, inplace=True)

df_energy.head()

,Country,ISO2,Indicator,Technology,Energy Type,Unit,2000,2001,2002,2003,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Advanced Economies,NaN,Electricity Generation,Bioenergy,Total Renewable,Gigawatt-hours (GWh),110053.796,106524.006,116718.930,124214.129,...,208500.849,224904.936,239773.166,252341.180,264181.433,274068.035,283583.581,287858.956,293917.663,296348.479
1,Advanced Economies,NaN,Electricity Installed Capacity,Bioenergy,Total Renewable,Megawatt (MW),18788.046,20371.893,21203.368,23002.811,...,41776.427,43690.635,46115.653,48575.397,50782.092,53205.387,55580.629,58470.495,59285.425,59698.238
2,Advanced Economies,NaN,Electricity Generation,Fossil fuels,Total Non-Renewable,Gigawatt-hours (GWh),5564064.840,5504484.424,5688040.566,5840829.880,...,6184941.257,6192108.018,6103628.918,5966341.171,5895296.029,5862708.121,5724991.355,5780558.599,5552191.520,5239862.113
3,Advanced Economies,NaN,Electricity Installed Capacity,Fossil fuels,Total Non-Renewable,Megawatt (MW),1241564.571,1285952.868,1351010.805,1397854.522,...,1548870.635,1549434.445,1536047.763,1543835.825,1520799.171,1507492.990,1504881.823,1519986.073,1496654.330,1487980.682
4,Advanced Economies,NaN,Electricity Generation,Other renewable,Total Renewable,Gigawatt-hours (GWh),26919.100,26474.100,27225.065,27854.065,...,37192.016,38076.418,38936.706,40294.245,40949.973,40915.479,41096.979,41910.671,41891.049,42477.806


In [4]:
df_gen = df_energy[df_energy['Indicator'] == 'Electricity Generation']
df_cap = df_energy[df_energy['Indicator'] == 'Electricity Installed Capacity']
df_world_gen = df_energy[(df_energy['Country'] == 'World') & (df_energy['Indicator'] == 'Electricity Generation')]
df_world_cap = df_energy[(df_energy['Country'] == 'World') & (df_energy['Indicator'] == 'Electricity Installed Capacity')]

# setting common colors
color_discrete_map={
                "Fossil fuels": px.colors.qualitative.Prism[10],
                "Hydropower": px.colors.qualitative.Prism[1],
                "Nuclear": px.colors.qualitative.Prism[7],
                "Wind energy": px.colors.qualitative.Prism[2],
                "Solar energy": px.colors.qualitative.Prism[6],
                "Other renewable": px.colors.qualitative.Prism[3],
                "Bioenergy": px.colors.qualitative.Prism[4],
                "Other non-renewable": px.colors.qualitative.Prism[0],
                }

# Renewable vs. Non-renewable energy generation
df_world_gtype = df_world_gen.groupby(['Energy Type']).sum()
df_world_gtype.reset_index(inplace=True)

df_world_gtype = pd.melt(df_world_gtype, id_vars=['Energy Type'], var_name='Year', value_name='Energy generation (GWH)')

#Renweable vs. Non-renewable energy capacity
df_world_ctype = df_world_cap.groupby(['Energy Type']).sum()
df_world_ctype.reset_index(inplace=True)

df_world_ctype = pd.melt(df_world_ctype, id_vars=['Energy Type'], var_name='Year', value_name='Energy capacity (MW)')

In [5]:
fig = px.line(df_world_gtype, 
              x='Year', 
              y='Energy generation (GWH)',  
              title='Renewable vs. Non-renewable energy generation',
              color='Energy Type',
              color_discrete_map={'Total Renewable': 'green', 'Total Non-Renewable': 'darkred'},
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()

In [6]:
df_world_gtype
df_wide = df_world_gtype.pivot(index='Year', columns='Energy Type', values='Energy generation (GWH)')
df_wide.reset_index(inplace=True)
df_wide.head()

Energy Type,Year,Total Non-Renewable,Total Renewable
0,2000,1.268707e+07,2845004.094
1,2001,1.286124e+07,2787635.099
2,2002,1.336986e+07,2881804.749
3,2003,1.392384e+07,2906547.980
4,2004,1.450968e+07,3117220.260


In [7]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Total Renewable'], 
        name = 'Total Renewable',
        mode = 'lines',
        line = dict(color = px.colors.qualitative.Prism[3]),
        fill = 'tozeroy'))
fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Total Non-Renewable'],
        name = 'Total Non-Renewable',
        line=dict(color= px.colors.qualitative.Prism[7]),
        mode = 'lines',
        fill = 'tonexty'))
fig.update_layout(
        title = "Renewable vs. Non-renewable energy generation", 
        width = 800, height = 600,
        #legend_title_text = 'Energy Type',
        )
fig.update_xaxes(
     title_text = 'Year',
     tickmode = 'linear',
     dtick = 2)
fig.update_yaxes(
     title_text = "Energy generation (GWH)")

fig.show()
py.plot(fig, filename = 'Renewable vs Non-Renewable generation', auto_open=False)

'https://plotly.com/~tom.hd/30/'

In [8]:
fig = px.line(df_world_ctype, 
              x='Year', 
              y='Energy capacity (MW)',  
              title='Renewable vs. Non-renewable energy capacity',
              color='Energy Type',
              color_discrete_map={'Renewable': 'green', 'Non-renewable': 'red'},
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()

In [9]:
world_tech_gen = df_world_gen.groupby(['Technology']).sum()
world_tech_gen.reset_index(inplace=True)
world_tech_gen = pd.melt(world_tech_gen, id_vars=['Technology'], var_name='Year', value_name='Energy generation (GWH)')
world_tech_gen.head()

,Technology,Year,Energy generation (GWH)
0,Bioenergy,2000,141835.806
1,Fossil fuels,2000,9951151.347
2,Hydropower,2000,2617799.578
3,Nuclear,2000,2588111.258
4,Other non-renewable,2000,147811.769


In [76]:
fig = px.line(world_tech_gen, 
              x='Year', 
              y='Energy generation (GWH)',  
              title='Global energy generation by technology',
              color='Technology',
              color_discrete_map=color_discrete_map,
              category_orders={'Technology': world_tech_gen.sort_values('Energy generation (GWH)', ascending=False)['Technology']},
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Global energy generation by technology', auto_open=False)

'https://plotly.com/~tom.hd/9/'

In [11]:
world_tech_gen
df_wide = world_tech_gen.pivot(index='Year', columns='Technology', values='Energy generation (GWH)')
df_wide.reset_index(inplace=True)
df_wide.head()

Technology,Year,Bioenergy,Fossil fuels,Hydropower,Nuclear,Other non-renewable,Other renewable,Solar energy,Wind energy
0,2000,141835.806,9.951151e+06,2617799.578,2588111.258,147811.769,53112.918,1331.432,30924.360
1,2001,142245.720,1.007019e+07,2553804.066,2636315.588,154735.978,52164.124,1700.326,37720.863
2,2002,155964.550,1.055034e+07,2618365.993,2661408.762,158110.394,53096.074,1976.666,52401.466
3,2003,167211.457,1.112921e+07,2619311.025,2637712.039,156917.172,54256.347,2400.086,63369.065
4,2004,182779.255,1.160646e+07,2790789.385,2740962.153,162255.617,56493.683,3082.361,84075.576


In [12]:
fig4 = go.Figure()

fig4.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Nuclear'],
        name = 'Nuclear',
        mode = 'lines',
        fill = 'tonexty'))
fig4.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Hydropower'],
        name = 'Hydropower',
        mode = 'lines',
        fill = 'tonexty'))
fig4.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Fossil fuels'], 
        name = 'Fossil fuels',
        mode = 'lines',
        fill = 'tonexty'))
fig4.update_layout(
        title = "Energy generation by technology", 
        width = 800, height = 600)

fig4.show()

In [13]:
world_tech_cap = df_world_cap.groupby(['Technology']).sum()
world_tech_cap.reset_index(inplace=True)
world_tech_cap = pd.melt(world_tech_cap, id_vars=['Technology'], var_name='Year', value_name='Energy capacity (MW)')

In [67]:
fig = px.line(world_tech_cap, 
              x='Year', 
              y='Energy capacity (MW)',  
              title='Global energy capacity by technology',
              color='Technology',
              color_discrete_map=color_discrete_map,
              category_orders={'Technology': world_tech_cap.sort_values('Energy capacity (MW)', ascending=False)['Technology']},
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Global capacity generation by technology', auto_open=False)

'https://plotly.com/~tom.hd/12/'

In [15]:
# checking for missing data
missing_percentages = df_world_cap.isna().mean() * 100
print(missing_percentages)

# taking the most recent data as no values are missing
world_tech_cap20 = world_tech_cap[world_tech_cap['Year'] == '2020']

Country          0.0
ISO2           100.0
Indicator        0.0
Technology       0.0
Energy Type      0.0
Unit             0.0
2000             0.0
2001             0.0
2002             0.0
2003             0.0
2004             0.0
2005             0.0
2006             0.0
2007             0.0
2008             0.0
2009             0.0
2010             0.0
2011             0.0
2012             0.0
2013             0.0
2014             0.0
2015             0.0
2016             0.0
2017             0.0
2018             0.0
2019             0.0
2020             0.0
dtype: float64


In [16]:
fig = px.bar(world_tech_cap20, 
              x='Energy capacity (MW)', 
              y='Technology',
              title='Global energy capacity in 2020',
              orientation='h',
              height=600,width=800,
              color='Technology',
              color_discrete_map=color_discrete_map,
              category_orders={'Technology': world_tech_cap20.sort_values('Energy capacity (MW)', ascending=False)['Technology']})

fig.show()
py.plot(fig, filename = 'Global energy capacity in 2020', auto_open=False)

'https://plotly.com/~tom.hd/14/'

In [61]:
# filtering for countries
df_countries = df_energy.dropna(subset=['ISO2'])
df_countries = df_countries[df_countries['Indicator'] == 'Electricity Generation']

# filtering for the top 10 countries by generation in 2020
country_gen = df_countries.groupby('Country').sum()['2020']
country_gen = country_gen.sort_values(ascending=False)
country_gen = country_gen.reset_index()
top_10_gen = country_gen.iloc[:10]

df_top_10 = df_countries[df_countries['Country'].isin(top_10_gen['Country'].tolist())]
df_top_10 = df_top_10.groupby(['Country', 'Technology']).sum()['2020']
df_top_10 = df_top_10.reset_index()
df_top_10.sort_values('2020', ascending=False, inplace=True)
df_top_10

,Country,Technology,2020
17,China,Fossil fuels,5194360.062
72,United States,Fossil fuels,2573324.000
18,China,Hydropower,1321710.000
41,India,Fossil fuels,1263058.250
74,United States,Nuclear,823150.000
...,...,...,...
63,Russia,Bioenergy,393.000
37,Germany,Other renewable,231.000
21,China,Other renewable,150.000
13,Canada,Other renewable,0.000


In [62]:
fig = px.bar(df_top_10, 
              x='Country', 
              y='2020',
              title='Top 10 energy generators in 2020',
              orientation='v',
              barmode='stack',
              height=600,width=900,
              color='Technology',
              color_discrete_map= color_discrete_map,
              )
fig.update_yaxes(
        title_text = "Energy generation (GWH)"
)          
fig.show()
py.plot(fig, filename = 'Top 10 energy generators', auto_open=False)

In [57]:
top10_sum = df_top_10.groupby('Country').sum()
top10_sum = top10_sum.reset_index()

top10_sum

,Country,2020
0,Brazil,621263.887
1,Canada,651896.000
2,China,7780166.491
3,France,531762.263
4,Germany,572666.000
5,India,1614455.148
6,Japan,1017779.000
7,Korea,578592.000
8,Russia,1089667.700
9,United States,4260044.011


In [20]:
df_gtype = df_gen.groupby(['Country', 'Technology']).sum()
df_gtype.reset_index(inplace=True)
df_gtype = pd.melt(df_gtype, id_vars=['Country', 'Technology'], var_name='Year', value_name='Energy generation (GWH)')
top_4 = df_gtype[df_gtype['Country'].isin(['China', 'India', 'United States', 'Russia'])]

In [38]:
df_ctype = df_cap.groupby(['Country', 'Technology']).sum()
df_ctype.reset_index(inplace=True)
df_ctype = pd.melt(df_ctype, id_vars=['Country', 'Technology'], var_name='Year', value_name='Energy capacity (MW)')
top_4 = df_ctype[df_ctype['Country'].isin(['China', 'India', 'United States', 'Russia'])]

In [39]:
fig = px.line(top_4, 
              x='Year', 
              y='Energy capacity (MW)',  
              title='Technology type of major energy generators',
              facet_col='Country',
              facet_col_wrap=2,
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=600,width=800)
fig.update_yaxes(matches=None, showticklabels=True, tickangle=0,)
fig.show()
py.plot(fig, filename = 'Technology type of major energy generators', auto_open=False)

'https://plotly.com/~tom.hd/18/'

In [42]:
world_tech_cap20

,Technology,Year,Energy capacity (MW)
160,Bioenergy,2020,1.328708e+05
161,Fossil fuels,2020,4.346246e+06
162,Hydropower,2020,1.211607e+06
163,Nuclear,2020,4.035416e+05
164,Other non-renewable,2020,1.510321e+05
165,Other renewable,2020,1.496110e+04
166,Solar energy,2020,7.172109e+05
167,Wind energy,2020,7.316232e+05


In [75]:

fig = px.line(df_ctype[df_ctype['Country'] == 'China'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='China\'s energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'China'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'China Energy Mix', auto_open=False)

'https://plotly.com/~tom.hd/57/'

In [72]:
fig = px.line(df_ctype[df_ctype['Country'] == 'United States'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='US energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'Unites States'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'US Energy Mix', auto_open=True)

'https://plotly.com/~tom.hd/82/'

In [73]:
fig = px.line(df_ctype[df_ctype['Country'] == 'India'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='India\'s energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'India'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'India Energy Mix', auto_open=True)

'https://plotly.com/~tom.hd/84/'

In [74]:
fig = px.line(df_ctype[df_ctype['Country'] == 'Russia'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='Russia\'s energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'Russia'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Russia Energy Mix', auto_open=True)

'https://plotly.com/~tom.hd/86/'

In [28]:
df_renweable = df_world_gen[df_world_gen['Energy Type'] == 'Total Renewable']
df_renweable = df_renweable.groupby(['Technology']).sum()
df_renweable.reset_index(inplace=True)
df_renweable = pd.melt(df_renweable, id_vars=['Technology'], var_name='Year', value_name='Energy generation (GWH)')
df_renweable.head()

,Technology,Year,Energy generation (GWH)
0,Bioenergy,2000,141835.806
1,Hydropower,2000,2617799.578
2,Other renewable,2000,53112.918
3,Solar energy,2000,1331.432
4,Wind energy,2000,30924.360


In [29]:
fig = px.line(df_renweable, 
              x='Year', 
              y='Energy generation (GWH)',  
              title='Renweable energy generation trend',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Renweable energy generation trend', auto_open=False)

'https://plotly.com/~tom.hd/24/'

In [30]:
df_renweable = df_world_cap[df_world_cap['Energy Type'] == 'Total Renewable']
df_renweable = df_renweable.groupby(['Technology']).sum()
df_renweable.reset_index(inplace=True)
df_renweable = pd.melt(df_renweable, id_vars=['Technology'], var_name='Year', value_name='Energy capacity (MW)')
df_renweable.head()

,Technology,Year,Energy capacity (MW)
0,Bioenergy,2000,29158.6980
1,Hydropower,2000,697170.0399
2,Other renewable,2000,8473.7000
3,Solar energy,2000,1227.6690
4,Wind energy,2000,16932.4830


In [31]:
fig = px.line(df_renweable, 
              x='Year', 
              y='Energy capacity (MW)',  
              title='Renweable energy capacity trend',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Renweable energy capacity trend', auto_open=False)

'https://plotly.com/~tom.hd/26/'